In [17]:
import numpy as np
from copy import deepcopy

In [18]:
class Grid:

    def __init__(self, parent=None):
        self.GRID_SIZE = (3, 3)
        self.create_grid()
        self.children = []
        self.results = []
        self.parent = parent
        self.curr_val=1
        self.base_val=1

    def create_grid(self):
        self.grid = np.zeros(self.GRID_SIZE)
    
    def is_leaf(self):
        return True if len(self.children)==0 else False
    
    def is_terminal(self):
        if self.score_grid()!="not finished":
            return True
        return False

    def available_positions(self):
        pos = np.where(self.grid==0)
        av_pos = [(x, y) for x, y in zip(pos[0], pos[1])]
        return av_pos
    
    def grid_complete(self):
        if len(self.available_positions())==0:
            return True
        return False
    
    def win_grid(self):

        grid = self.grid
        diag = np.diag(self.grid)
        reverse_diag = np.diag(np.fliplr(self.grid))

        if diag[0]==diag[1]==diag[2]!=0:
            return diag[0]
        
        if reverse_diag[0]==reverse_diag[1]==reverse_diag[2]!=0:
            return reverse_diag[0]
        
        for col in range(3):
            if grid[0, col]==grid[1, col]==grid[2, col]!=0:
                return grid[0, col]
        
        for row in range(3):
            if grid[row, 0]==grid[row, 1]==grid[row, 2]!=0:
                return grid[row, 0]
        
        return False
    
    def play_move(self, pos:tuple):
        if len(pos)!=2:
            print("position length should be 2")
            return "position length should be 2"
        self.grid[pos]=self.curr_val
        self.curr_val= -self.curr_val

        
    def get_children(self):
        if not self.children:
            self.children = [(lambda g: (g.play_move(pos), g)[1])(deepcopy(self)) for pos in self.available_positions()]
            for child in self.children:
                child.parent = self
        return self.children

    def score_grid(self):
        win = self.win_grid()

        if win==self.base_val:
            return 100
        if win==-self.base_val:
            return -100
        if self.grid_complete():
            return 0
        return "not finished"
    
    def reset_grid(self):
        self.children = []
        self.results = []
        self.parent = None

    def show_grid(self):
        grid = self.grid.astype(str)
        grid[grid == "0.0"] = " "
        grid[grid == "1.0"] = "X"
        grid[grid == "-1.0"] = "O"
        
        # Afficher la grille avec une délimitation claire
        print("-------------")
        for i, row in enumerate(grid):
            print(f"| {' | '.join(row)} |")
            print("-------------")



In [19]:
def ucb_score(child:Grid, parent:Grid, C=np.sqrt(2)):
    if len(child.results)==0:
        return np.inf
    return np.mean(child.results) + C*np.sqrt(np.log(len(parent.results))/len(child.results))


In [20]:
def selection(grid:Grid):
    if not grid.is_leaf():
        ucbs = [ucb_score(child, grid) for child in grid.get_children()]
        candidate = np.random.choice(np.flatnonzero(ucbs==np.max(ucbs)))
        candidate = grid.get_children()[candidate]
        return selection(candidate)
    return grid

In [21]:
def expansion(grid:Grid):
    if not grid.is_terminal():
        if len(grid.results)==0 and grid.parent is not None:
            return grid
        grid.children = grid.get_children()
        return grid.children[np.random.randint(len(grid.children))]
    return grid

In [22]:
def playout(grid:Grid):
    playout_node = deepcopy(grid)
    while not playout_node.is_terminal():
        av_pos = playout_node.available_positions()
        pos = av_pos[np.random.randint(len(av_pos))]
        playout_node.play_move(pos)
    reward = playout_node.score_grid()
    del playout_node
    return reward

In [23]:
def backpropagation(grid:Grid, reward):
    if grid.parent is None:
        grid.results.append(reward)
        return "Done"
    grid.results.append(reward)
    backpropagation(grid.parent, -reward)

In [95]:
def get_action(grid:Grid, value=1):
    for i in range(len(grid.available_positions())):
        leaf = selection(grid)
        expanded = expansion(leaf)
        for iter_playout in range(1000):
            reward = playout(expanded)
            _ = backpropagation(expanded, value*reward)
    
    ucbs = [ucb_score(child, grid) for child in grid.children]
    candidate_id = np.random.choice(np.flatnonzero(ucbs==np.max(ucbs)))
    return grid.children[candidate_id]


In [96]:
grid = Grid()
new_grid = deepcopy(grid)

while True:

    new_grid = get_action(new_grid,1)
    new_grid.show_grid()
    new_grid.reset_grid()

    new_grid = get_action(new_grid,-1)
    new_grid.show_grid()
    new_grid.reset_grid()

-------------
|   |   |   |
-------------
|   | X |   |
-------------
|   |   |   |
-------------
-------------
|   |   | O |
-------------
|   | X |   |
-------------
|   |   |   |
-------------
-------------
| X |   | O |
-------------
|   | X |   |
-------------
|   |   |   |
-------------
-------------
| X |   | O |
-------------
|   | X |   |
-------------
|   |   | O |
-------------
-------------
| X |   | O |
-------------
|   | X | X |
-------------
|   |   | O |
-------------
-------------
| X |   | O |
-------------
| O | X | X |
-------------
|   |   | O |
-------------
-------------
| X | X | O |
-------------
| O | X | X |
-------------
|   |   | O |
-------------
-------------
| X | X | O |
-------------
| O | X | X |
-------------
|   | O | O |
-------------
-------------
| X | X | O |
-------------
| O | X | X |
-------------
| X | O | O |
-------------


ValueError: zero-size array to reduction operation maximum which has no identity